<a href="https://colab.research.google.com/github/RenatodaCostaSantos/Machine-Learning---Lessons/blob/main/Supervised%20ML/Logistic%20regression/Building_LR_model_lesson_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a logistic regression model

It is finally time to apply what we learned in the previous lessons. We will build a logistic regression model and evaluate it on the test set and check how well our models will perform. That will guide us to go back and begin to iterate it and improve its predictive power.

We will briefly introduce concepts such as sparsity, regularization (overfitting) and discuss multiclass classification problems. 

Let's start by loading the libraries and the auto dataset once more.



In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
auto = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Logistic regression/automobiles.csv')

In [3]:
auto.head(8)

,symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
1,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
2,1,158,audi,gas,std,four,sedan,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,17710
3,1,158,audi,gas,turbo,four,sedan,fwd,front,105.8,...,131,mpfi,3.13,3.40,8.3,140,5500,17,20,23875
4,2,192,bmw,gas,std,two,sedan,rwd,front,101.2,...,108,mpfi,3.50,2.80,8.8,101,5800,23,29,16430
5,0,192,bmw,gas,std,four,sedan,rwd,front,101.2,...,108,mpfi,3.50,2.80,8.8,101,5800,23,29,16925
6,0,188,bmw,gas,std,two,sedan,rwd,front,101.2,...,164,mpfi,3.31,3.19,9.0,121,4250,21,28,20970
7,0,188,bmw,gas,std,four,sedan,rwd,front,101.2,...,164,mpfi,3.31,3.19,9.0,121,4250,21,28,21105


In [4]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          159 non-null    int64  
 1   normalized_losses  159 non-null    int64  
 2   make               159 non-null    object 
 3   fuel_type          159 non-null    object 
 4   aspiration         159 non-null    object 
 5   num_of_doors       159 non-null    object 
 6   body_style         159 non-null    object 
 7   drive_wheels       159 non-null    object 
 8   engine_location    159 non-null    object 
 9   wheel_base         159 non-null    float64
 10  length             159 non-null    float64
 11  width              159 non-null    float64
 12  height             159 non-null    float64
 13  curb_weight        159 non-null    int64  
 14  engine_type        159 non-null    object 
 15  num_of_cylinders   159 non-null    object 
 16  engine_size        159 non

In [5]:
auto.describe()

,symboling,normalized_losses,wheel_base,length,width,height,curb_weight,engine_size,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
count,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000
mean,0.735849,121.132075,98.264151,172.413836,65.607547,53.899371,2461.138365,119.226415,3.300126,3.236352,10.161132,95.836478,5113.836478,26.522013,32.081761,11445.729560
std,1.193086,35.651285,5.167416,11.523177,1.947883,2.268761,481.941321,30.460791,0.267336,0.294888,3.889475,30.718583,465.754864,6.097142,6.459189,5877.856195
min,-2.000000,65.000000,86.600000,141.100000,60.300000,49.400000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,15.000000,18.000000,5118.000000
25%,0.000000,94.000000,94.500000,165.650000,64.000000,52.250000,2065.500000,97.000000,3.050000,3.105000,8.700000,69.000000,4800.000000,23.000000,28.000000,7372.000000
50%,1.000000,113.000000,96.900000,172.400000,65.400000,54.100000,2340.000000,110.000000,3.270000,3.270000,9.000000,88.000000,5200.000000,26.000000,32.000000,9233.000000
75%,2.000000,148.000000,100.800000,177.800000,66.500000,55.500000,2809.500000,135.000000,3.560000,3.410000,9.400000,114.000000,5500.000000,31.000000,37.000000,14719.500000
max,3.000000,256.000000,115.600000,202.600000,71.700000,59.800000,4066.000000,258.000000,3.940000,4.170000,23.000000,200.000000,6600.000000,49.000000,54.000000,35056.000000


As in the previous lessons, we will create a feature called 'high_price' and that will be our target column:

In [6]:
# Create high_price column
auto['high_price'] = 0
auto.loc[auto['price'] > 15000, 'high_price'] = 1

In [7]:
# Check for high_price column values
auto['high_price'].value_counts()

0    119
1     40
Name: high_price, dtype: int64

In [8]:
# Create features dataframe
X = auto.drop(['price','high_price'], axis = 1)
# Create target
y = auto['high_price']

In [9]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123, stratify = y)

# Building two different models


In order to give an idea of a machine learning workflow, we will build two different models by using two different features of our dataset and compare their accuracy first on the train set:

In [10]:
# Define features for both models
X1 = X_train[['horsepower','length']]
X2 = X_train[['compression_ratio','stroke']]

In [11]:
from sklearn.linear_model import LogisticRegression

# Instantiate two models
model1 = LogisticRegression()
model2 = LogisticRegression()

# Fit the models
model1.fit(X1,y_train)
model2.fit(X2,y_train)

LogisticRegression()

In [12]:
# Testing the accuracy of models on the training set
accuracy_1 = model1.score(X1,y_train)
print(f'The accuracy for model1 on the train set was: {accuracy_1*100:.2f}%')

accuracy_2 = model2.score(X2,y_train)
print(f'The accuracy for model2 on the train set was: {accuracy_2*100:.2f}%')

The accuracy for model1 on the train set was: 87.40%
The accuracy for model2 on the train set was: 72.44%


As we can see, model1 performed much better on the training set, with 87.4% of correct predictions, while model2 had an accuracy of 72.4%.

# Feature selection: Split-Apply-Combine

So far, we've worked out how to build a logistic regression model and how to compute its accuracy given some features. However, when building a model, we would like to increase our chances to find the best model. We chose some random features above and saw that different features have a different impact on the accuracy of the model. In a realistic machine learning workflow, one important goal is to try to find the best set of features. This task is known as **feature selection**.

In linear regression, one can use visualizations or compute Pearson's r coefficient in order to select features that are linearly correlated with the target column. That helps the model to increase its performance. We can implement a similar idea in a logistic regression model.

In a logistic regression model, we can check if a feature will help the model to identify the outcomes more precisely by following the steps:

1 - We **split** the dataset into two datasets by using the target column, as shown below:

![split](https://raw.githubusercontent.com/RenatodaCostaSantos/Machine-Learning---Lessons/main/Supervised%20ML/Logistic%20regression/images/dataframe_split.png)

2 - We aggregate the data using a specific feature and compute a quantity that can help us summarize the data for that feature, for example, the mean value for that column in both datasets. This procedure can be visualized in the figure below:

![mean](https://raw.githubusercontent.com/RenatodaCostaSantos/Machine-Learning---Lessons/main/Supervised%20ML/Logistic%20regression/images/mean_agg.png)

3 - The last step is to **combine** both outcomes in a single dataframe as we can see in the figure below:

![combine](https://raw.githubusercontent.com/RenatodaCostaSantos/Machine-Learning---Lessons/main/Supervised%20ML/Logistic%20regression/images/combine.png)


If the values for the means are very sparse, the distribution of values for that feature is significantly different for different values of the outcomes. That will help the model to make better predictions. 

We can implement the procedure above with the groupby() and agg() methods in Pandas. For example, groupby ('high_price') will split the dataset into two different dataframes, one containing only 0 values for the target variable, and one containing only 1 values. That's the first step of the procedure we described above.

The next step is implemented with the agg() method that takes a dictionary containing the column names we want to select as keys and the operation we want to execute as values. The final result is combined and shown in a dataframe. These are steps 2 and 3 described above. 

Let's use this procedure to understand why the model1 had better accuracy than the model2 in the previous section.





In [13]:
# Perform the split-apply-combine in pandas
auto.groupby('high_price').agg(
    {
        'horsepower': 'mean',
        'compression_ratio': 'mean',
        'stroke': 'mean',
        'length': 'mean'
    }
)

,horsepower,compression_ratio,stroke,length
high_price,,,,
0,83.907563,9.886723,3.241261,168.22605
1,131.325000,10.977500,3.221750,184.87250


The values for the means for 'horsepower' and 'length' are significantly more different on different outcomes than the 'compression_ratio' and 'stroke' features. That explains why model1 was more accurate than model2 in the previous section.

# Model complexity

As we saw in the previous section, choosing features wisely is very important. If we decide to increase the number of features of a model, it will tend to become very good at predicting values on the training set, but it will struggle to find important patterns once we present new data to the model. That is the intuition behind overfitting. Too many features and the model can perform well on the training set and fail to be accurate in the test set. 

Let's build three different models of increasing complexity and evaluate them on the training and test sets in order to develop an intuition for the sentence above:

In [14]:
# Define training features for three different models
X1 = X_train[['horsepower']]
X2 = X_train[['horsepower','compression_ratio']]
X3 = X_train[['horsepower','compression_ratio','city_mpg']]

# Define test features for three different models
X1_test = X_test[['horsepower']]
X2_test = X_test[['horsepower','compression_ratio']]
X3_test = X_test[['horsepower','compression_ratio','city_mpg']]

In [15]:
# Instantiate three models
model1 = LogisticRegression()
model2 = LogisticRegression()
model3 = LogisticRegression()

In [16]:
# Fit models on train sets
model1.fit(X1,y_train)
model2.fit(X2,y_train)
model3.fit(X3,y_train)

LogisticRegression()

In [17]:
# Compute accuracy on training set
accuracy_1_train = model1.score(X1,y_train)
accuracy_2_train = model2.score(X2,y_train)
accuracy_3_train = model3.score(X3,y_train)

In [18]:
print(f'The accuracies for the three models on the training set were: {accuracy_1_train*100:.2f}%, {accuracy_2_train*100:.2f}% and, {accuracy_3_train*100:.2f}%.')

The accuracies for the three models on the training set were: 87.40%, 88.19% and, 91.34%.


As we can see, there is an increasing trend in the accuracies as we increase the model's complexity by adding more features. It is becoming better at finding the outcomes in the training set. However, let's check if it holds on the test set:

In [19]:
# Compute accuracy on test set
accuracy_1_test = model1.score(X1_test,y_test)
accuracy_2_test = model2.score(X2_test,y_test)
accuracy_3_test = model3.score(X3_test,y_test)

In [20]:
print(f'The accuracies for the three models on the test set were: {accuracy_1_test*100:.2f}%, {accuracy_2_test*100:.2f}% and, {accuracy_3_test*100:.2f}%.')

The accuracies for the three models on the test set were: 84.38%, 81.25% and, 90.62%.


This result illustrates that the increasing trend in the training set does not hold in the test set predictions. In fact, the model's performance decreased as we introduced the second feature and increased again once the third feature was introduced. 

The main message here is that we should be careful when assessing the training set results.

# Multi-class classification

Imagine that instead of a binary classification problem we had multiple categorical values in the target variable. That would make it a multi-class classification problem. Let's create a multi-class target column in the auto dataset before discussing what would change in a machine-learning workflow.


In [21]:
# Create a multi-class target variable
auto['price_category'] = 'Low'
auto.loc[(auto['price'] >= 10000) & (auto['price'] < 15000), 'price_category'] = 'Mid'
auto.loc[auto['price'] >= 15000, 'price_category'] = 'High'

We had three different values for the outcomes, 'Low', 'Mid', and 'High'. What would change in the machine learning workflow in this case? 

The good news is that the LogisticRegression class from sklearn deals with it without any issues. Almost all methods are available for this problem. The accuracy as a metric is still well-defined. However, sensitivity and specificity only make sense in a binary classification problem, so they are not available in multi-class classification problems.

In a multi-class classification problem, we can use the confusion matrix, which generalizes the concepts of sensitivity and specificity.

![confusion](https://raw.githubusercontent.com/RenatodaCostaSantos/Machine-Learning---Lessons/main/Supervised%20ML/Logistic%20regression/images/confusion.png)


It will always be a square $n \times n$ matrix, where $n$ is the number of different outcomes. The off-diagonal terms are values that were incorrectly predicted by the model. These values can inform us if the model is having trouble identifying one class while doing fine in others.

Let's implement the machine-learning workflow in a multi-class classification problem:

In [22]:
# Split dataset into feature and target
X = auto.drop(['price','price_category'],axis = 1)
y = auto['price_category']

In [23]:
# Define train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 415)

In [25]:
# Define a model using just a few features
X1 = X_train[['engine_size','horsepower']]
X1_test = X_test[['engine_size','horsepower']]

In [28]:
from sklearn.metrics import confusion_matrix

# Instantiate a model
model = LogisticRegression()

# Fit the model
model.fit(X1,y_train)

# Find accuracy of the model on test set
accuracy = model.score(X1_test, y_test)
print(f'The accuracy of the model in the test set was: {accuracy*100:.2f}%')

# Find test predictions
predictions = model.predict(X1_test)

# Calculate confusion matrix
confusion = confusion_matrix(y_test, predictions)

print(confusion)

The accuracy of the model in the test set was: 65.62%
[[ 8  3  1]
 [ 0 13  0]
 [ 3  4  0]]


According to the accuracy and confusion matrix, the model is not very accurate and is having more trouble identifying high price cars.

# Comunicating results

When we finish a project, we have to be clear about the process used while manipulating the data. Here are a few non-exhaustive lists of questions we should be able to answer after finishing a project:

- Which fractions of the data were used for the training and test sets?

- Did we perform any transformations on the data? If so, why was that necessary/relevant?

- Which features did we use to build the model? What is the reason we chose those features?

- How did the model perform on new data?

- Which metric did you choose to make those predictions? Why did you choose that metric?

- Does the model perform better on some specific label? Which one?

These questions are not only useful to other people to make decisions when using your model, but also to guide yourself on your choices while building the model itself.


# Summary

In this lesson we:

- Built two logistic regression models using different features and checked their accuracy.

- Used the split-apply-combine feature selection method to understand why one of the models was more accurate than the other.

- Built three models with different levels of complexity and use them to discuss overfitting.

- Built a multi-class classification model and introduced the confusion matrix metric.

- Showed good practice communication skills when reporting your model to others.